## Library

In [1]:
import requests
import random
import pandas as pd
import time
import math
from copy import deepcopy
from statistics import mean

## Notebook

#### Requisição

In [2]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

#### Tratamento de Dados

In [3]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                    if (key in [product for product in json[i] if ('prod' in product and (int(product[5]) < 9 and len(product) < 7))]):
                        json[i][key] = {'qnt':[round(qnt)], 'price':None}
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None}
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

#### Gerar valores aleatórios

In [4]:
def generate_random_price(n_min=1.0, n_max=100.0) -> float:
    return round(random.uniform(n_min, n_max), 2)

#### Precificação

##### Função logistica -> A mesma postada no doc pelo professor

In [5]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / (1 + math.e ** -v))

##### Função para calcular a variação -> A mesma postada no doc pelo professor

In [6]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

##### Função que calcula o preço que será utilizado na semana seguinte

In [7]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['next_week_price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

##### Função que atualiza o banco de dados (onde consta a quantidade de vendas da última semana e o preço dos produtos) e aplica o preço nos produtos do JSON

In [8]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None
                    product_db[key]['next_week_price'] = product_db[key]['price']              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['next_week_price'])
                    product_db[key]['price'] = product_db[key]['next_week_price']
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

##### Função que formata o DataFrame

In [9]:
def df_to_all_df(json, all_df, week):
    for transacao in json:
        json_to_df = deepcopy(transacao)
        date = json_to_df['date']
        id = json_to_df['id']
        del json_to_df['date']
        del json_to_df['id']
        df = pd.DataFrame(json_to_df).T
        df['id'] = id
        df['date'] = date
        df['month'] = pd.to_datetime(df['date']).dt.to_period('M')
        df['week'] = week
        df = pd.concat([all_df, df])
        # df=df.set_index(['date','id'])

    return df

In [10]:
def format_all_df(all_df:pd.DataFrame):
    all_df=all_df.reset_index().rename(columns={'index':'product'}).set_index('id').reset_index()
    all_df['qnt'] = [float(item[0]) for item in all_df['qnt']]
    return all_df

In [18]:
def monthy_sales_csv(all_df:pd.DataFrame):
    monthy_sales_csv = pd.DataFrame(all_df.groupby(['month', 'product', 'id']).qnt.sum())
    monthy_sales_csv.to_csv('monthy_sales.csv')
    return monthy_sales_csv

#### Main

In [12]:
product_database = {}
weeks = 8
all_df = pd.DataFrame([])

for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    all_df = df_to_all_df(json=json, all_df=all_df, week=week)
        
all_df=format_all_df(all_df=all_df)
all_df.to_csv('allsales.csv')

In [13]:
all_df.head()

,id,product,qnt,price,date,month,week
0,lafwmplv9z5tl4w5ucu,prod_1,46.0,55.96,Thu Mar 5 01:45:30 2020,2020-03,0
1,lafwmplv9z5tl4w5ucu,prod_2,32.0,47.56,Thu Mar 5 01:45:30 2020,2020-03,0
2,lafwmplv9z5tl4w5ucu,prod_6,32.0,64.04,Thu Mar 5 01:45:30 2020,2020-03,0
3,lafwmplv9z5tl4w5ucu,prod_7,17.0,20.47,Thu Mar 5 01:45:30 2020,2020-03,0
4,lafwmplv9z5tl4w5ucu,prod_8,296.0,80.48,Thu Mar 5 01:45:30 2020,2020-03,0


In [14]:
all_df.tail()

,id,product,qnt,price,date,month,week
48,lafwmqeorz4mvo99vv,prod_15,604.205179,578.34,Wed Apr 15 20:46:00 2020,2020-04,6
49,lafwmqjj8wvunug2qh,prod_7,10.000000,233.166083,Wed Apr 22 21:38:22 2020,2020-04,7
50,lafwmqjj8wvunug2qh,prod_9,327.772775,784.472344,Wed Apr 22 21:38:22 2020,2020-04,7
51,lafwmqjj8wvunug2qh,prod_10,121.700669,758.387813,Wed Apr 22 21:38:22 2020,2020-04,7
52,lafwmqjj8wvunug2qh,prod_15,350.031938,867.51,Wed Apr 22 21:38:22 2020,2020-04,7


In [19]:
monthy_sales_csv(all_df)

qnt
month   product id                              
2020-03 prod_0  lafwmpp4rjbt6ckwj2r    17.000000
        prod_1  lafwmplv9z5tl4w5ucu    46.000000
                lafwmpnmcgoqbfgpr6o    10.000000
                lafwmpu7bohyy3jrras     7.000000
        prod_10 lafwmpu7bohyy3jrras   752.093391
        prod_11 lafwmplv9z5tl4w5ucu     0.000000
                lafwmpnmcgoqbfgpr6o   695.675569
                lafwmpp4rjbt6ckwj2r   407.412896
                lafwmpu7bohyy3jrras   681.831121
        prod_15 lafwmplv9z5tl4w5ucu   331.815130
                lafwmpnmcgoqbfgpr6o   586.282462
                lafwmpp4rjbt6ckwj2r   390.944596
                lafwmpu7bohyy3jrras    80.030692
        prod_2  lafwmplv9z5tl4w5ucu    32.000000
                lafwmpnmcgoqbfgpr6o    43.000000
        prod_4  lafwmpnmcgoqbfgpr6o    31.000000
                lafwmpp4rjbt6ckwj2r    15.000000
                lafwmpu7bohyy3jrras     7.000000
        prod_5  lafwmpnmcgoqbfgpr6o    40.000000
                lafwmpp4rjbt6ckwj2r    22.000000
                lafwmpu7bohyy3jrras    19.000000
        prod_6  lafwmplv9z5tl4w5ucu    32.000000
                lafwmpp4rjbt6ckwj2r    35.000000
        prod_7  lafwmplv9z5tl4w5ucu    17.000000
                lafwmpp4rjbt6ckwj2r    19.000000
                lafwmpu7bohyy3jrras     8.000000
        prod_8  lafwmplv9z5tl4w5ucu   296.000000
                lafwmpp4rjbt6ckwj2r   359.000000
                lafwmpu7bohyy3jrras   672.000000
        prod_9  lafwmplv9z5tl4w5ucu     0.000000
                lafwmpnmcgoqbfgpr6o   686.771245
                lafwmpu7bohyy3jrras   808.220628
2020-04 prod_0  lafwmpxxk8ipt42fj4     29.000000
        prod_1  lafwmpxxk8ipt42fj4     25.000000
                lafwmq7ie8fgqjl308t    46.000000
        prod_10 lafwmqjj8wvunug2qh    121.700669
        prod_11 lafwmqeorz4mvo99vv    829.057938
        prod_15 lafwmpxxk8ipt42fj4     61.229754
                lafwmq7ie8fgqjl308t   364.074283
                lafwmqeorz4mvo99vv    604.205179
                lafwmqjj8wvunug2qh    350.031938
        prod_3  lafwmq7ie8fgqjl308t    48.000000
        prod_4  lafwmq7ie8fgqjl308t    15.000000
                lafwmqeorz4mvo99vv     33.000000
        prod_5  lafwmqeorz4mvo99vv     39.000000
        prod_6  lafwmqeorz4mvo99vv     28.000000
        prod_7  lafwmqjj8wvunug2qh     10.000000
        prod_8  lafwmpxxk8ipt42fj4    488.000000
                lafwmq7ie8fgqjl308t   773.000000
                lafwmqeorz4mvo99vv   1011.000000
        prod_9  lafwmpxxk8ipt42fj4     21.809091
                lafwmqeorz4mvo99vv     23.203083
                lafwmqjj8wvunug2qh    327.772775